In [ ]:
# En este notebook se hizo una restructuración de la entrega previa. 
# Se procedió a hacer una limpia de las variables relevantes para las preguntas. 
# Sin embargo, todavía no se construyen las consultas que responden las preguntas. 
# Esperamos que mañana tengamos la respuesta a todas las preguntas de la etapa 1-3
# Podemos observar que algunas preguntas más generales se pueden responder, como: 
# ¿Cuáles son las distintas rutas en todos los años?
# ¿Cuáles son los productos de todos los años? 

In [95]:
import pandas as pd
import numpy as np
import sys
import glob
import os.path

In [71]:
folder_name = "./data/"

data_path = os.path.join(folder_name,'*xlsx')

files = glob.glob(data_path)

In [72]:
files

['./data/v2017.xlsx',
 './data/v2012.xlsx',
 './data/v2013.xlsx',
 './data/v2015.xlsx',
 './data/v2018.xlsx',
 './data/v2014.xlsx',
 './data/v2016.xlsx']

In [73]:
years = []
for file in files:
    df = pd.read_excel(file, sheet_name = None)
    year = file.replace('./data/v', '')
    year = year.replace('.xlsx', '')
    years.append([df, year])
    df['Hoja1'].to_pickle(year)
    

In [75]:
def extract_product_tables(df): 
    cols = ['PRODUCTO', 'CARGA', 'EXISTENCIA', 'DEVOLUCION', 'VENTA TOTAL', 'PRECIO * PRODUC', '$', 'VENTA EN LT', 'RUTA', 'DIA', 'SEMANA']
    productos = pd.DataFrame(columns= cols)
    test_copy = df
    for i in range(int(df.shape[0])):
        for j in range(int(df.shape[1])):
            if type(df.iloc[i, j]) == unicode and ('RUTA:' in df.iloc[i, j] or 'IVAN' in df.iloc[i, j]):
                ruta = ''
                dia = ''
                semana = ''
                ruta = df.iloc[i, j]
                if type(df.iloc[i - 1,j]) == unicode:
                    dia = df.iloc[i - 1, j]
                if type(df.iloc[i - 2,j]) == unicode and 'SEMANA' in df.iloc[i - 2,j]:
                    semana = df.iloc[i - 2, j]
                new_entry = test_copy.iloc[i + 2:i + 10, j:j + 12].T
                new_entry['RUTA'] = ruta
                new_entry['DIA'] = dia
                new_entry['SEMANA'] = semana
                new_entry.columns = cols
                if ruta != '' and not 'QUESO' in ruta:
                    productos = productos.append(new_entry, ignore_index=True)
    return productos

In [76]:
cols = ['PRODUCTO', 'CARGA', 'EXISTENCIA', 'DEVOLUCION', 'VENTA TOTAL', 'PRECIO * PRODUC', '$', 'VENTA EN LT', 'RUTA', 'DIA', 'SEMANA', 'ANIO']
productos = pd.DataFrame(columns= cols)
for year in years:
    year_raw = year[0]['Hoja1']
    year_name = year[1]
    year_products = extract_product_tables(year_raw)
    year_products['ANIO'] = year_name
    productos = productos.append(year_products, ignore_index=True)

In [77]:
productos.shape

(90919, 12)

In [78]:
productos.to_pickle('productos')

In [137]:
productos = pd.read_pickle('productos')

In [ ]:
# LIMPIAA!

In [138]:
# Queremos eliminar todas las observaciones que no son rutas
# Maldito Ivan.
rutas = []
for ruta in productos['RUTA']:
    if not ruta in rutas:
        rutas.append(ruta)
rutas

[u'RUTA: FERMIN',
 u'RUTA: JUGUERAS',
 u'RUTA: SAN MARCOS',
 u'RUTA:MARTIN',
 u'RUTA: MARTIN',
 u'RUTA: ERICK',
 u'RUTA: JESUS',
 u'RUTA: TRADICION',
 u'RUTA IVAN',
 u'QHR IVAN',
 u'QH IVAN',
 u'RUTA IVAN ',
 u'BALANCE SEMANAL IVAN ',
 u'IVAN ']

In [139]:
productos = productos.loc[productos['RUTA'] != 'QHR IVAN']
productos = productos.loc[productos['RUTA'] != 'QH IVAN']
productos = productos.loc[productos['RUTA'] != 'BALANCE SEMANAL IVAN ']
productos = productos.loc[productos['RUTA'] != 'IVAN ']
rutas = []
for ruta in productos['RUTA']:
    if not ruta in rutas:
        rutas.append(ruta)
rutas

[u'RUTA: FERMIN',
 u'RUTA: JUGUERAS',
 u'RUTA: SAN MARCOS',
 u'RUTA:MARTIN',
 u'RUTA: MARTIN',
 u'RUTA: ERICK',
 u'RUTA: JESUS',
 u'RUTA: TRADICION',
 u'RUTA IVAN',
 u'RUTA IVAN ']

In [140]:
# Ahora redefinimos repetidos: 
productos = productos.replace('RUTA:MARTIN', 'RUTA: MARTIN')
productos = productos.replace('RUTA IVAN ', 'RUTA: IVAN')
productos = productos.replace('RUTA IVAN', 'RUTA: IVAN')
rutas = []
for ruta in productos['RUTA']:
    if not ruta in rutas:
        rutas.append(ruta)
rutas

[u'RUTA: FERMIN',
 u'RUTA: JUGUERAS',
 u'RUTA: SAN MARCOS',
 'RUTA: MARTIN',
 u'RUTA: ERICK',
 u'RUTA: JESUS',
 u'RUTA: TRADICION',
 'RUTA: IVAN']

In [141]:
productos_nombres = []
for producto in productos['PRODUCTO']:
    if not producto in productos_nombres:
        productos_nombres.append(producto)
productos_nombres

[u'PRODUCTO8',
 u'PRODUCTO3',
 u'PRODUCTO7',
 u'PRODUCTO2',
 u'PRODUCTO1',
 u'PRODUCTO5',
 u'PRODUCTO9',
 nan,
 nan,
 nan,
 nan,
 u'PRODUCTO',
 u'PRODUCTO10',
 u'Queso Hebra',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 u'Queso fresco',
 u'Queso a\xf1ejo',
 u'Queso panela',
 u'PRODUCTO4',
 u'Requeson',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 u'PRODUCTO1 bolsa',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [142]:
# Elmininamos nan's en productos
productos = productos[productos['PRODUCTO'].isnull() == False]
productos_nombres = []
for producto in productos['PRODUCTO']:
    if not producto in productos_nombres:
        productos_nombres.append(producto)
productos_nombres

[u'PRODUCTO8',
 u'PRODUCTO3',
 u'PRODUCTO7',
 u'PRODUCTO2',
 u'PRODUCTO1',
 u'PRODUCTO5',
 u'PRODUCTO9',
 u'PRODUCTO',
 u'PRODUCTO10',
 u'Queso Hebra',
 u'Queso fresco',
 u'Queso a\xf1ejo',
 u'Queso panela',
 u'PRODUCTO4',
 u'Requeson',
 u'PRODUCTO1 bolsa',
 6,
 8,
 u'Queso hebra',
 u'LTRS',
 u'TOTAL QUESO',
 50,
 10.6,
 55.3,
 12.5,
 41.629999999999995,
 45.78,
 44.25,
 45.05,
 0,
 49.8,
 57.6,
 11.95,
 40,
 77.1,
 21.6,
 89.7,
 15,
 49.55,
 49.08,
 u'MS',
 67.8,
 52.8,
 25,
 74.5,
 u'TOTAL QUES',
 5.3,
 58.5,
 13.3,
 7.5,
 73.5,
 10.8,
 52.5,
 8.5,
 11.6,
 42.7,
 2.7,
 14.9,
 75,
 4.4,
 15.8,
 44.9,
 2.8,
 59.5,
 2.6,
 5,
 57.4,
 41.5,
 4.6,
 42.3,
 29.4,
 16.6,
 62.4,
 2.5,
 45,
 8.7,
 u'L-G-I',
 0.3,
 41.6,
 37.5,
 36,
 39.5,
 30,
 43.7,
 5.4,
 50.1,
 27.9,
 43,
 54.5,
 21,
 58.92,
 22.5,
 70.9,
 34.8,
 45.3,
 54.9,
 58.099999999999994,
 4,
 0.4,
 49.6,
 52,
 3.5,
 50.5,
 2,
 43.2,
 56.8,
 46.05,
 89.2,
 62.7,
 61,
 48.78,
 26.400000000000002,
 51.900000000000006,
 1.96,
 60,
 25.1

In [145]:
# Eliminamos productos invalidos
productos['PRODUCTO'] = productos['PRODUCTO'].apply(unicode)
productos = productos[productos['PRODUCTO'].str.contains("PRO") | productos['PRODUCTO'].str.contains("QUES") | productos['PRODUCTO'].str.contains("Ques")]
productos_nombres = []
for producto in productos['PRODUCTO']:
    if not producto in productos_nombres:
        productos_nombres.append(producto)
productos_nombres

[u'PRODUCTO8',
 u'PRODUCTO3',
 u'PRODUCTO7',
 u'PRODUCTO2',
 u'PRODUCTO1',
 u'PRODUCTO5',
 u'PRODUCTO9',
 u'PRODUCTO',
 u'PRODUCTO10',
 u'Queso Hebra',
 u'Queso fresco',
 u'Queso a\xf1ejo',
 u'Queso panela',
 u'PRODUCTO4',
 u'PRODUCTO1 bolsa',
 u'Queso hebra',
 u'TOTAL QUESO',
 u'TOTAL QUES']

In [147]:
# Estandarizamos nombres
productos = productos.replace('PRODUCTO1 bolsa', 'PRODUCTO1')
productos = productos.replace('TOTAL QUES', 'TOTAL QUESO')
productos_nombres = []
for producto in productos['PRODUCTO']:
    if not producto in productos_nombres:
        productos_nombres.append(producto)
productos_nombres

[u'PRODUCTO8',
 u'PRODUCTO3',
 u'PRODUCTO7',
 u'PRODUCTO2',
 u'PRODUCTO1',
 u'PRODUCTO5',
 u'PRODUCTO9',
 u'PRODUCTO',
 u'PRODUCTO10',
 u'Queso Hebra',
 u'Queso fresco',
 u'Queso a\xf1ejo',
 u'Queso panela',
 u'PRODUCTO4',
 u'Queso hebra',
 u'TOTAL QUESO']

In [35]:
# Eliminamos todas las observaciones que sean en realidad los nombres de las columnas
productos = productos.loc[productos['CARGA'] != 'CARGA']

In [150]:
dias = []
for dia in productos['DIA']:
    if not dia in dias:
        dias.append(dia)
dias

['',
 u'SABADO',
 u'DIA:LUNES ',
 u'DIA LUNES ',
 u'DIA DOMIN',
 u'DIA MARTES ',
 u'DIA MERTES ',
 u'MIERCOLES',
 u'DIA MIERCOLES ',
 u'DIA MIERCO',
 u'jueves',
 u'DIA JUEVES  ',
 u'DIA JUEVES ',
 u' DIA VIERNES ',
 u'DIA VIERNES ',
 u'miercoles',
 u'martes',
 u'DIA: SABADO',
 u'DIA: LUNES',
 u'DIA: MARTES',
 u'DIA: MIERCOLES',
 u'DIA: JUEVES',
 u'DIA: VIERNES',
 u'DIA: SABADO 07 07 2012',
 u'DIA: SABADO 07-07-12',
 u'DIA:LUNES  09-07-2012',
 u'DIA: LUNES 9-07-12',
 u'DIA:LUNES      09/07/12',
 u'DIA: LUNES-DOMINGO 08 Y 09 /07/12',
 u'DIA: DOMINGO 08-07-12',
 u'DIA:MARTES 10-07-12',
 u'DIA: MARTES 26-06-12',
 u'DIA: MARTES 10-07-12',
 u'DIA: MIERCOLES 11-07-12',
 u'DIA:MIERCOLES 11/07/12',
 u'DIA:JUEVES        12 /07/12',
 u'DIA: JUEVES 12-07-12',
 u'DIA:VIERNES      13/07/12',
 u'DIA: VIERNES 13-07-12',
 u'DIA: SABADO 14-07-12',
 u'DIA:LUNES  16-07-2012',
 u'DIA: LUNES 16-06-12',
 u'DIA:LUNES  16-07-12',
 u'DIA: LUNES 16-07-12',
 u'DIA: DOMINGO 15-07-12',
 u'DIA:MARTES 17-07-12',
 u'D

In [158]:
# Filtramos aquellas observaciones que claramente no son dias
productos = productos[productos['DIA'].str.contains("SAB") | productos['DIA'].str.contains("DOM") | 
                         productos['DIA'].str.contains("LUN") | productos['DIA'].str.contains("MAR") | 
                         productos['DIA'].str.contains("MIER") | productos['DIA'].str.contains("JUE") | 
                         productos['DIA'].str.contains("VIE") | productos['DIA'].str.contains("sab") | 
                         productos['DIA'].str.contains("dom") | productos['DIA'].str.contains("lun") | 
                         productos['DIA'].str.contains("mar") | productos['DIA'].str.contains("mier") | 
                         productos['DIA'].str.contains("jue") | productos['DIA'].str.contains("vie")]
dias = []
for dia in productos['DIA']:
    if not dia in dias:
        dias.append(dia)
dias

[u'SABADO',
 u'DIA:LUNES ',
 u'DIA LUNES ',
 u'DIA DOMIN',
 u'DIA MARTES ',
 u'MIERCOLES',
 u'DIA MIERCOLES ',
 u'DIA MIERCO',
 u'jueves',
 u'DIA JUEVES  ',
 u'DIA JUEVES ',
 u' DIA VIERNES ',
 u'DIA VIERNES ',
 u'miercoles',
 u'martes',
 u'DIA: SABADO',
 u'DIA: LUNES',
 u'DIA: MARTES',
 u'DIA: MIERCOLES',
 u'DIA: JUEVES',
 u'DIA: VIERNES',
 u'DIA: SABADO 07 07 2012',
 u'DIA: SABADO 07-07-12',
 u'DIA:LUNES  09-07-2012',
 u'DIA: LUNES 9-07-12',
 u'DIA:LUNES      09/07/12',
 u'DIA: LUNES-DOMINGO 08 Y 09 /07/12',
 u'DIA: DOMINGO 08-07-12',
 u'DIA:MARTES 10-07-12',
 u'DIA: MARTES 26-06-12',
 u'DIA: MARTES 10-07-12',
 u'DIA: MIERCOLES 11-07-12',
 u'DIA:MIERCOLES 11/07/12',
 u'DIA:JUEVES        12 /07/12',
 u'DIA: JUEVES 12-07-12',
 u'DIA:VIERNES      13/07/12',
 u'DIA: VIERNES 13-07-12',
 u'DIA: SABADO 14-07-12',
 u'DIA:LUNES  16-07-2012',
 u'DIA: LUNES 16-06-12',
 u'DIA:LUNES  16-07-12',
 u'DIA: LUNES 16-07-12',
 u'DIA: DOMINGO 15-07-12',
 u'DIA:MARTES 17-07-12',
 u'DIA: MARTES 17-07-12',


In [159]:
# Estandarizamos los dias:
dias_real = []
for dia in dias:
    if "SAB" in dia:
        dia = 'SABADO'
        dias_real.append(dia)
    if "DOM" in dia:
        dia = 'DOMINGO'
        dias_real.append(dia)
    if "LUN" in dia:
        dia = 'LUNES'
        dias_real.append(dia)
    if "MAR" in dia:
        dia = 'MARTES'
        dias_real.append(dia)
    if "MIE" in dia:
        dia = 'MIERCOLES'
        dias_real.append(dia)
    if "JUE" in dia:
        dia = 'JUEVES'
        dias_real.append(dia)
    if "VIE" in dia:
        dia = 'VIERNES'
        dias_real.append(dia)
    if "sab" in dia:
        dia = 'SABADO'
        dias_real.append(dia)
    if "dom" in dia:
        dia = 'DOMINGO'
        dias_real.append(dia)
    if "lun" in dia:
        dia = 'LUNES'
        dias_real.append(dia)
    if "mar" in dia:
        dia = 'MARTES'
        dias_real.append(dia)
    if "mie" in dia:
        dia = 'MIERCOLES'
        dias_real.append(dia)
    if "jue" in dia:
        dia = 'JUEVES'
        dias_real.append(dia)
    if "vie" in dia:
        dia = 'VIERNES'
        dias_real.append(dia)

for i in range(len(dias)):
    productos = productos.replace(dias[i], dias_real[i])

dias = []
for dia in productos['DIA']:
    if not dia in dias:
        dias.append(dia)
dias

['SABADO', 'LUNES', 'DOMINGO', 'MARTES', 'MIERCOLES', 'JUEVES', 'VIERNES']

In [160]:
semanas = []
for semana in productos['SEMANA']:
    if not semana in semanas:
        semanas.append(semana)
semanas

['',
 u'SEMANA DEL 24 AL 30 DE AGOSTO DEL 2013 ',
 u'SEMANA DEL 07 AL 13 DE JULIO DEL 2012',
 u'SEMANA DEL 14 AL 20 DE JULIO DEL 2012',
 u'SEMANA DEL 21 AL 27 DE JULIO 2012',
 u'SEMANA DE L21 AL 27 JULIO 2012',
 u'SEMANA DEL  21 AL 27 DE JULIIO 2012',
 u'SEMANA DEL 21 AL 237 DE JULIO 2012',
 u'SEMANA DEL 21 AL 27  DE JULIO 2012',
 u'SEMANA DEL 21 AL 27 2012',
 u'SEMANA DEL 21 AL 27 DE JULIO DE 2102',
 u'SEMANA DEL 28 DE JULIO AL 3 DE AGOSTO 2012',
 u'SEMANA DE L 4 AGOTO AL 10  2012',
 u'SEMANA DEL 4 AL 10 AGOSTO 2012',
 u'SEMANA DEL 4 AL 10 AGOSTO 2012 ',
 u'SEMANA  DEL 4  AL 10 AGOSTO 2012',
 u'SEMANA DEL 4  AL 10 AGOSTO 2012',
 u'SEMANA DEL 4 AL 10 DE AGOSTO 2012',
 u'SEMANA  DE 4  AL 10 AGOSTO 2012',
 u'SEMANA DEL 4 AL 10  AGOSTO 2012',
 u'SEMANA DEL 4 AL 10 AGSTO  2012',
 u'SEMANA DEL 4 A L 10 AGOSTO 2012',
 u'SEMANA DEL 4 AL 10 DE AGOSTO 21402',
 u'SEMANA DEL 40AL 10 AGOSTO 2012 ',
 u'SEMANA DEL 4 AL 10 AGOSTO 2102 ',
 u'SEMANA DEL 4 AL 10  AGOSTO 2012 ',
 u'SEMANA DEL 4 0AL 10 AG

In [58]:
copy_todo = productos
rutas = []
for ruta in copy_todo['RUTA']:
    while ruta[0] == ' ':
        ruta = ruta[1:]
    while '  ' in ruta:
        ruta = ruta.replace('  ', ' ')
    ruta = ruta.replace(': ', ':')
    if not ruta in rutas:
        rutas.append(ruta)
rutas

[u'RUTA:FERMIN',
 u'RUTA:SAN MARCOS',
 u'RUTA:JUGUERAS',
 u'RUTA:MARTIN',
 u'RUTA:ERICK',
 u'RUTA:JESUS']

In [63]:
productos.loc[' RUTA:TRADICION' == productos['RUTA']]

,PRODUCTO,CARGA,EXISTENCIA,DEVOLUCION,VENTA TOTAL,PRECIO * PRODUC,$,VENTA EN LT,RUTA,DIA,SEMANA,ANIO


In [9]:
productos.loc[productos['CARGA'] == 'CARGA']

,PRODUCTO,CARGA,EXISTENCIA,DEVOLUCION,VENTA TOTAL,PRECIO * PRODUC,$,VENTA EN LT,RUTA,DIA,SEMANA
11,PRODUCTO,CARGA,EXISTENCIA,DEVOLUCION,VENTA TOTAL,PRECIO * PRODUC,$,VENTA EN LT,RUTA: FERMIN,,SEMANA DEL 31 AL 5 ENERO 2017
23,PRODUCTO,CARGA,EXISTENCIA,DEVOLUCION,VENTA TOTAL,PRECIO * PRODUC,$,VENTA EN LT,RUTA: JUGUERAS,SABADO,
47,PRODUCTO,CARGA,EXISTENCIA,DEVOLUCION,VENTA TOTAL,PRECIO * PRODUC,$,VENTA EN LT,RUTA: FERMIN,,
59,PRODUCTO,CARGA,EXISTENCIA,DEVOLUCION,VENTA TOTAL,PRECIO * PRODUC,$,VENTA EN LT,RUTA: JUGUERAS,DIA:LUNES,
95,PRODUCTO,CARGA,EXISTENCIA,DEVOLUCION,VENTA TOTAL,PRECIO * PRODUC,$,VENTA EN LT,RUTA: FERMIN,DIA MARTES,
107,PRODUCTO,CARGA,EXISTENCIA,DEVOLUCION,VENTA TOTAL,PRECIO * PRODUC,$,VENTA EN LT,RUTA: JUGUERAS,DIA MERTES,
131,PRODUCTO,CARGA,EXISTENCIA,DEVOLUCION,VENTA TOTAL,PRECIO * PRODUC,$,VENTA EN LT,RUTA: FERMIN,MIERCOLES,
143,PRODUCTO,CARGA,EXISTENCIA,DEVOLUCION,VENTA TOTAL,PRECIO * PRODUC,$,VENTA EN LT,RUTA: JUGUERAS,DIA MIERCOLES,
167,PRODUCTO,CARGA,EXISTENCIA,DEVOLUCION,VENTA TOTAL,PRECIO * PRODUC,$,VENTA EN LT,RUTA: FERMIN,jueves,
179,PRODUCTO,CARGA,EXISTENCIA,DEVOLUCION,VENTA TOTAL,PRECIO * PRODUC,$,VENTA EN LT,RUTA: JUGUERAS,DIA JUEVES,
